In [6]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [7]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("WordCount").setMaster("local[*]")
sc = SparkContext(conf=conf)


# Set the file path to the local directory
file_path = "fradulent_emails.txt"  # Ensure the file is in the same folder as this script
lines = sc.textFile(file_path)

# Splits lines into words and maps each word to (word, 1)
def processLine(line):
    words = line.split(" ")
    return [(word.lower(), 1) for word in words]

# Sum the counts for each word ie reduce by key
def sumWordCounts(a, b):
    return a + b

# Sort word counts in descending order to easily get top 20
def sortDesc(pair):
    return -pair[1]

# Process lines, count words, and get the top 20
myWords = lines.flatMap(processLine)
wordCounts = myWords.reduceByKey(sumWordCounts)
top20Words = wordCounts.takeOrdered(21, key=sortDesc)
# Got top 21 words because the most used "word" is : , which is not actually a word

# Print the results
for word, count in top20Words:
    print(f"{word} : {count}")



/Users/aaronrusk/Desktop/anaconda3/lib/python3.12/site-packages/pyspark/./bin/spark-submit: line 21: dirname: command not found
/Users/aaronrusk/Desktop/anaconda3/lib/python3.12/site-packages/pyspark/./bin/spark-submit: line 21: /find-spark-home: No such file or directory
/Users/aaronrusk/Desktop/anaconda3/lib/python3.12/site-packages/pyspark/./bin/spark-submit: line 27: /bin/spark-class: No such file or directory
/Users/aaronrusk/Desktop/anaconda3/lib/python3.12/site-packages/pyspark/./bin/spark-submit: line 27: exec: /bin/spark-class: cannot execute: No such file or directory


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [5]:
sc.stop()

NameError: name 'sc' is not defined

The most frequently used words in order are:
the, to, of, and, i, in, you, this, a, my, your, for, will, that, as, is, be, with, from, me

This count does not really give any indication that they are taken from phishing emails, they are simply 20 of what I would predict are some of the most commonly used words in the English language. To improve the word count tool, we could purposely omit these super common conjuctions, pronouns, etc. that would be common in any English text. We could have a list of these common words and compare the words we find in the emails, omitting the words that also appear in that list. Then what would be left are the most used uncommon words, which would be a better indicator of if an email is likely to be a scam.